# Move Ion Reference data into an MPContribs Project

---
## Header

#### Imports

In [25]:
from pprint import pprint
from monty.serialization import loadfn, dumpfn
from pymatgen.core.ion import Ion

---
## Main Code

### Load the ion reference data from the .json file and examine its format

`aqeuous_ions.json` can be downloaded from https://github.com/materialsproject/pymatpro/blob/master/pymatpro/pourbaix_tools/aqueous_ions.json

In [24]:
ion_ref_data = loadfn("aqueous_ions.json")

In [13]:
# NOTE: all energy units are eV/formula unit
ion_ref_data[0]

{'Major_Elements': ['Li'],
 'Source': 'H. E. Barner and R. V. Scheuerman, Handbook of thermochemical data for compounds and aqueous species, Wiley, New York (1978)',
 'Reference Solid': 'Li2O',
 'Reference solid energy': -5.8167501960259,
 'Energy': -3.0441,
 'Name': 'Li[+]'}

### Path one entry according to `patch_aqueous_ions.json` on pymatpro
https://github.com/materialsproject/pymatpro/blob/master/pymatpro/pourbaix_tools/patch_aqueous_ions.json


In [14]:
for i, e in enumerate(ion_ref_data):
    if e["Name"] == "HGaO2[2-]":
        ion_ref_data[i] = {
            "Energy": -7.1099,
            "Major_Elements": [
                "Ga"
            ],
            "Name": "HGaO3[2-]",
            "Reference Solid": "Ga2O3",
            "Reference solid energy": -10.347724703224722,
            "Source": "D. D. Wagman et al., Selected values for inorganic and C1 and C2 Organic substances in SI units, The NBS table of chemical thermodynamic properties, Washington (1982)"
        }
        print("Replaced entry at index {}".format(i))

Replaced entry at index 121


In [15]:
ion_ref_data[121]

{'Energy': -7.1099,
 'Major_Elements': ['Ga'],
 'Name': 'HGaO3[2-]',
 'Reference Solid': 'Ga2O3',
 'Reference solid energy': -10.347724703224722,
 'Source': 'D. D. Wagman et al., Selected values for inorganic and C1 and C2 Organic substances in SI units, The NBS table of chemical thermodynamic properties, Washington (1982)'}

### Initialize the MPContribs client

In [16]:
from mpcontribs.client import Client
name = 'ion_ref_data' # this should be your project, see from the project URL
client = Client("iBryZ5c2YfcQpbE8IFWwGRH29VSKhJPH")

### Reshape the data

In [17]:
ion_contribs =[]

for d in ion_ref_data:
    ret = {}
    ret["project"] = name
    ret["is_public"] = True
    ret["identifier"] = d["Name"]
    ret["data"] = {}
    ret["data"]["charge"] = Ion.from_formula(d["Name"]).charge
    ret["data"]["ΔGᶠ"] = "{:.5g} kJ/mol".format(d["Energy"]*96.485) # convert from eV/f.u. to kJ/mol
    ret["data"]["MajElements"] = d["Major_Elements"][0]
    ret["data"]["RefSolid"] = d["Reference Solid"]
    ret["data"]["ΔGᶠRefSolid"] = "{:.4g} kJ/mol".format(d["Reference solid energy"]*96.485)# convert from eV/f.u. to kJ/mol
    ret["data"]["reference"] = d["Source"]
    
    ion_contribs.append(ret)

In [18]:
# did we miss any data?
len(ion_ref_data) - len(ion_contribs)

0

In [26]:
pprint(ion_contribs[30])

{'data': {'MajElements': 'Na',
          'RefSolid': 'Na2O',
          'charge': 1.0,
          'reference': 'H. E. Barner and R. V. Scheuerman, Handbook of '
                       'thermochemical data for compounds and aqueous species, '
                       'Wiley, New York (1978)',
          'ΔGᶠ': '-261.86 kJ/mol',
          'ΔGᶠRefSolid': '-376.3 kJ/mol'},
 'identifier': 'Na[+]',
 'is_public': True,
 'project': 'ion_ref_data'}


### Update project references, column descriptions, etc.

In [17]:
client.projects.update_entry(
    pk="ion_ref_data", project={"other": 
                                         {"formula": "Chemical formula of the aqueous species",
                                          "charge": "Charge on the aqueous species",
                                          "ΔGᶠ": "Gibbs free energy of formation of the aqueous species from the elements",
                                          "MajElementsᶠ": None,
                                          "MajElements": "Elements contained in the aqueous species",
                                         "RefSolid": "Solid compound to which the aqueous species energy is referenced",
                                          "ΔGᶠRefSolid": "Gibbs free energy of formation of the reference solid compound",
                                          "Ion": None
                                         },
                                "description": "This project contains experimental ion dissolution energies that are used by pymatgen when constructing Pourbaix diagrams. See the Persson2012 reference for a detailed description of the thermodynamic framework used."
                                        }
).result()

{'description': 'This project contains experimental ion dissolution energies that are used by pymatgen when constructing Pourbaix diagrams. See the Persson2012 reference for a detailed description of the thermodynamic framework used.',
 'other': {'formula': 'Chemical formula of the aqueous species',
  'charge': 'Charge on the aqueous species',
  'ΔGᶠ': 'Gibbs free energy of formation of the aqueous species from the elements',
  'MajElements': 'Elements contained in the aqueous species',
  'RefSolid': 'Solid compound to which the aqueous species energy is referenced',
  'ΔGᶠRefSolid': 'Gibbs free energy of formation of the reference solid compound'}}

In [19]:
client.projects.update_entry(
    pk="ion_ref_data", project={"authors": "Various authors (see references). Data compiled by the Materials Project team."
                                        }
).result()

{'authors': 'Various authors (see references). Data compiled by the Materials Project team.'}

In [20]:
client.projects.update_entry(
    pk="ion_ref_data", project={"title": "Aqueous Ion Reference Data"
                                        }
).result()

{'title': 'Aqueous Ion Reference Data'}

In [21]:
client.projects.update_entry(
    pk="ion_ref_data", project={"references": [
        {"label":"Persson2012", 'url':"https://doi.org/10.1103/PhysRevB.85.235438"},
        {'label': 'NBS1', 'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-1.pdf'},
    {'label': 'NBS2', 'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-2.pdf'},
    {'label': 'NBS3', 'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-3.pdf'},
    {'label': 'NBS4', 'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-4.pdf'},
    {'label': 'NBS5', 'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-5.pdf'},
    {'label': 'NBS6', 'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-6.pdf'},
    {'label': 'NBS7', 'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-7.pdf'},
    {'label': 'NBS8', 'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-8.pdf'},
    {"label":"Pourbaix", 'url':"https://www.worldcat.org/title/atlas-of-electrochemical-equilibria-in-aqueous-solutions/oclc/563921897"}]
                                        }
).result()

{'references': [{'label': 'Persson2012',
   'url': 'https://doi.org/10.1103/PhysRevB.85.235438'},
  {'label': 'NBS1',
   'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-1.pdf'},
  {'label': 'NBS2',
   'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-2.pdf'},
  {'label': 'NBS3',
   'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-3.pdf'},
  {'label': 'NBS4',
   'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-4.pdf'},
  {'label': 'NBS5',
   'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-5.pdf'},
  {'label': 'NBS6',
   'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-6.pdf'},
  {'label': 'NBS7',
   'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-7.pdf'},
  {'label': 'NBS8',
   'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-8.pdf'},
  {'label': 'Pourbaix',
   'url': 'https://www.worldcat.org/title/atla

### Check the project

In [3]:
client.get_project(name)

{'name': 'ion_ref_data',
 'is_public': True,
 'title': 'Aqueous Ion Reference Data',
 'owner': 'RKingsbury@lbl.gov',
 'is_approved': True,
 'unique_identifiers': True,
 'long_title': 'Aqueous Ion Reference Data',
 'authors': 'Various authors (see references). Data compiled by the Materials Project team.',
 'description': 'This project contains experimental ion dissolution energies that are used by pymatgen when constructing Pourbaix diagrams. See the Persson2012 reference for a detailed description of the thermodynamic framework used.',
 'references': [{'label': 'Persson2012',
   'url': 'https://doi.org/10.1103/PhysRevB.85.235438'},
  {'label': 'NBS1',
   'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-1.pdf'},
  {'label': 'NBS2',
   'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-2.pdf'},
  {'label': 'NBS3',
   'url': 'https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote270-3.pdf'},
  {'label': 'NBS4',
   'url': 'https://nvlpubs.

### Submit to MPContribs

In [22]:
# need to delete contributions first due to unique_identifiers=False
client.delete_contributions(name)
client.submit_contributions(ion_contribs, per_page=10, skip_dupe_check=True)
#client.contributions.create_entries(contributions=ion_contribs[0:100]).result()

### Appendix
----

In [27]:
break

SyntaxError: 'break' outside loop (<ipython-input-27-6aaf1f276005>, line 1)

### Note that `Ion.reduced_formula` generates the same formula for a few distinct entries in the database, e.g. "Fe2(OH)2[4+]" vs. "FeOH[2+]"

In [20]:
[e["Name"] for e in ion_ref_data if "Fe" in e["Name"]]

['Fe[2+]',
 'Fe[3+]',
 'FeO2[2-]',
 'FeOH[+]',
 'FeOH[2+]',
 'HFeO2[-]',
 'Fe(OH)2[+]',
 'Fe(OH)3(aq)',
 'Fe2(OH)2[4+]',
 'FeO2[-]',
 'FeO4[2-]']

In [21]:
[Ion.from_formula(e["Name"]).reduced_formula for e in ion_ref_data if "Fe" in e["Name"]]

['Fe[2+]',
 'Fe[3+]',
 'FeO2[2-]',
 'FeHO[+]',
 'FeHO[2+]',
 'FeHO2[-]',
 'Fe(HO)2[+]',
 'Fe(HO)3(aq)',
 'FeHO[2+]',
 'FeO2[-]',
 'FeO4[2-]']

In [22]:
[e for e in ion_ref_data if e["Name"] == "Fe2(OH)2[4+]"]

[{'Major_Elements': ['Fe'],
  'Source': 'D. D. Wagman et al., Selected values of chemical thermodynamic properties, NBS Technical note 270, Washington; 1968-1971',
  'Reference Solid': 'Fe2O3',
  'Reference solid energy': -7.685050670886141,
  'Energy': -4.84285,
  'Name': 'Fe2(OH)2[4+]'}]

In [23]:
[e for e in ion_ref_data if e["Name"] == "FeOH[2+]"]

[{'Major_Elements': ['Fe'],
  'Source': 'D. D. Wagman et al., Selected values of chemical thermodynamic properties, NBS Technical note 270, Washington; 1968-1971',
  'Reference Solid': 'Fe2O3',
  'Reference solid energy': -7.685050670886141,
  'Energy': -2.37954,
  'Name': 'FeOH[2+]'}]